In [21]:
from PIL import Image

# Convert mm to pixels for 300 DPI
def mm_to_px(mm, dpi=300):
    return int((mm / 25.4) * dpi)

# Card dimensions in mm
card_width_mm = 54
card_height_mm = 80

# Convert card dimensions to pixels
card_width_px = mm_to_px(card_width_mm)
card_height_px = mm_to_px(card_height_mm)

# Print out the dimensions for verification
print(f"Card dimensions in pixels: {card_width_px} x {card_height_px}")


Card dimensions in pixels: 637 x 944


In [22]:
# New card dimensions with additional 4mm padding on each side
additional_padding_mm = 4
new_card_width_mm = card_width_mm + additional_padding_mm
new_card_height_mm = card_height_mm + additional_padding_mm

# Convert new card dimensions to pixels for 300 DPI
new_card_width_px = mm_to_px(new_card_width_mm)
new_card_height_px = mm_to_px(new_card_height_mm)

# Print out the new dimensions for verification
print(f"New card dimensions with padding in pixels: {new_card_width_px} x {new_card_height_px}")


New card dimensions with padding in pixels: 685 x 992


In [23]:
from PIL import Image, ImageDraw

# Convert mm to pixels for 300 DPI
def mm_to_px(mm, dpi=300):
    return int((mm / 25.4) * dpi)

# Original card dimensions in mm
card_width_mm = 54
card_height_mm = 80
additional_padding_mm = 4

# New card dimensions with 4mm padding on each side
new_card_width_mm = card_width_mm + additional_padding_mm
new_card_height_mm = card_height_mm + additional_padding_mm

# Convert new card dimensions to pixels
new_card_width_px = mm_to_px(new_card_width_mm)
new_card_height_px = mm_to_px(new_card_height_mm)
card_width_px = mm_to_px(card_width_mm)
card_height_px = mm_to_px(card_height_mm)

# Load the .webp image
image_path = "/home/ravik/src_git/LP/projects/create_catan_cards/back.webp" # Replace with your image path
image = Image.open(image_path)

# Create a blank card with the new dimensions (white background)
card_with_padding = Image.new('RGB', (new_card_width_px, new_card_height_px), 'white')

# Get the dimensions of the image
image_width, image_height = image.size

# Calculate the position to center the image on the original card area
x_offset = (card_width_px - image_width) // 2 + (new_card_width_px - card_width_px) // 2
y_offset = (card_height_px - image_height) // 2 + (new_card_height_px - card_height_px) // 2

# Paste the image onto the card (centered on the original card area)
card_with_padding.paste(image, (x_offset, y_offset))

# Draw a dark border around the original card area
draw = ImageDraw.Draw(card_with_padding)
border_thickness = 5  # Adjust the border thickness as needed
border_color = (50, 50, 50)  # Dark gray color

# Draw the border (inside the larger card but around the original card size)
draw.rectangle(
    [(new_card_width_px - card_width_px) // 2, (new_card_height_px - card_height_px) // 2,
     (new_card_width_px + card_width_px) // 2, (new_card_height_px + card_height_px) // 2],
    outline=border_color, width=border_thickness
)

# Draw a dotted black line as a cutting guide
dotted_line_color = (0, 0, 0)  # Black
dotted_line_spacing = 5  # Spacing between dots
for x in range((new_card_width_px - card_width_px) // 2, (new_card_width_px + card_width_px) // 2, dotted_line_spacing * 2):
    draw.line([(x, (new_card_height_px - card_height_px) // 2), (x + dotted_line_spacing, (new_card_height_px - card_height_px) // 2)], fill=dotted_line_color, width=1)
    draw.line([(x, (new_card_height_px + card_height_px) // 2), (x + dotted_line_spacing, (new_card_height_px + card_height_px) // 2)], fill=dotted_line_color, width=1)

for y in range((new_card_height_px - card_height_px) // 2, (new_card_height_px + card_height_px) // 2, dotted_line_spacing * 2):
    draw.line([((new_card_width_px - card_width_px) // 2, y), ((new_card_width_px - card_width_px) // 2, y + dotted_line_spacing)], fill=dotted_line_color, width=1)
    draw.line([((new_card_width_px + card_width_px) // 2, y), ((new_card_width_px + card_width_px) // 2, y + dotted_line_spacing)], fill=dotted_line_color, width=1)

# Save the card to verify the output visually
card_with_padding.save("back_card_with_border.png", dpi=(300, 300))

# Optionally, show the card image (if you're using a local environment)
card_with_padding.show()


In [24]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from PIL import Image

# Convert mm to points for ReportLab (1 mm = 2.83465 points)
def mm_to_pt(mm_value):
    return mm_value * 2.83465

# Card dimensions with padding (in mm)
card_width_mm = 58
card_height_mm = 84

# Convert card dimensions to points for the PDF
card_width_pt = mm_to_pt(card_width_mm)
card_height_pt = mm_to_pt(card_height_mm)

# Number of cards per page based on available space
cards_per_row = 3
cards_per_column = 3
cards_per_page = cards_per_row * cards_per_column

# Total number of cards (in this case, 25)
total_cards = 100

# Generate PDF with proper handling of images and empty space
def generate_pdf():
    c = canvas.Canvas("back_catan_cards.pdf", pagesize=A4)
    
    card_image_path = "/home/ravik/src_git/LP/projects/create_catan_cards/back_card_with_border.png"  # Replace with actual card image

    for i in range(total_cards):
        # Calculate row and column for each card
        row = (i % cards_per_page) // cards_per_row
        col = i % cards_per_row

        # X and Y position for the card on the A4 page
        x_pos = mm_to_pt(10) + col * card_width_pt  # Adding 10mm as the margin
        y_pos = A4[1] - (mm_to_pt(10) + row * card_height_pt + card_height_pt)  # Start from the top

        # Draw the card image onto the PDF
        c.drawImage(card_image_path, x_pos, y_pos, card_width_pt, card_height_pt, mask='auto')

        # If we have placed enough cards for one page or reach the end of the set, create a new page
        if (i + 1) % cards_per_page == 0 and (i + 1) < total_cards:
            c.showPage()

    c.save()
    print("PDF generated successfully!")

# Call the function to generate the PDF
generate_pdf()


Opening in existing browser session.
PDF generated successfully!
